In [23]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import argrelextrema
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
from datetime import datetime, timedelta
import os
from IPython.display import display, HTML
from tqdm import tqdm
import plotly.graph_objects as go

In [24]:


dataframes = {}

list_of_candidate=[]

timeframe='4h'
outdir=f"{os.getcwd()}/data/{timeframe}"


for filename in os.listdir(outdir):
    # symbol = filename.replace(f"_{timeframe}.csv","")
    symbol = filename.split("_")[0]
    df = pd.read_csv(f"{outdir}/{filename}", index_col='Time')
    # if len(df) > 17:

# for filename in os.listdir('datasets'):
    #print(filename)
    # symbol = filename.split(".")[0]
    #print(symbol)
    # df = pandas.read_csv('datasets/{}'.format(filename))
    if df.empty:
        continue

    df['20sma'] = df['Close'].rolling(window=20).mean()
    df['stddev'] = df['Close'].rolling(window=20).std()
    df['lower_band'] = df['20sma'] - (2 * df['stddev'])
    df['upper_band'] = df['20sma'] + (2 * df['stddev'])

    df['TR'] = abs(df['High'] - df['Low'])
    df['ATR'] = df['TR'].rolling(window=20).mean()

    df['lower_keltner'] = df['20sma'] - (df['ATR'] * 1.5)
    df['upper_keltner'] = df['20sma'] + (df['ATR'] * 1.5)

    def in_squeeze(df):
        return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']

    df['squeeze_on'] = df.apply(in_squeeze, axis=1)

    # if (df.iloc[-3]['squeeze_on']) and (not df.iloc[-1]['squeeze_on']):
    #     print("{} is coming out the squeeze".format(symbol))
    #     list_of_candidate.append(symbol)

    # if df.iloc[-1]['squeeze_on'] and df.iloc[-2]['squeeze_on'] and df.iloc[-3]['squeeze_on'] and df.iloc[-4]['squeeze_on'] and df.iloc[-5]['squeeze_on'] and df.iloc[-1]['squeeze_on']:
    #     print("{} is in the squeeze".format(symbol))


    counter=0
    n=33
    for i in range(1, n+1):
        if df.iloc[-i]['squeeze_on']:
            counter+=1
    if counter == n:
        print("{} is in the squeeze".format(symbol))



    # save all dataframes to a dictionary
    # we can chart individual names below by calling the chart() function
    dataframes[symbol] = df




def chart(df):
    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
    upper_band = go.Scatter(x=df.index, y=df['upper_band'], name='Upper Bollinger Band', line={'color': 'red'})
    lower_band = go.Scatter(x=df.index, y=df['lower_band'], name='Lower Bollinger Band', line={'color': 'red'})

    upper_keltner = go.Scatter(x=df.index, y=df['upper_keltner'], name='Upper Keltner Channel', line={'color': 'blue'})
    lower_keltner = go.Scatter(x=df.index, y=df['lower_keltner'], name='Lower Keltner Channel', line={'color': 'blue'})

    fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_keltner, lower_keltner])
    # fig.layout.xaxis.type = 'category'
    fig.layout.xaxis.rangeslider.visible = False
    fig.show()


%store list_of_candidate



ASRUSDT is in the squeeze
AUCTIONUSDT is in the squeeze
CVPUSDT is in the squeeze
DFUSDT is in the squeeze
DNTUSDT is in the squeeze
DREPUSDT is in the squeeze
GHSTUSDT is in the squeeze
LAZIOUSDT is in the squeeze
MBOXUSDT is in the squeeze
MFTUSDT is in the squeeze
PHAUSDT is in the squeeze
SANTOSUSDT is in the squeeze
TCTUSDT is in the squeeze
Stored 'list_of_candidate' (list)


In [26]:
df = dataframes['TCTUSDT'][-20:]
chart(df)
